# Project 5 Classification Group E

Randy , Rylan , Alex

If you would like to see the output of the code, please see the other individual files uploaded 

### First bit, download and clean up 

Note, this code is written for keeling not google colab (needed extra processors to get it to run more quickly).

In [ ]:
from metpy.io import parse_metar_to_dataframe
import pandas as pd
import wget

from tqdm import tqdm
import os
import datetime
import warnings
import glob 

%pylab inline


def padder(x):
    if x < 10:
        x = '0' + str(x)
    else:
        x = str(x)

    return x

def download_par(item):
    try:
        ff=wget.download(item,'./raw_data/'+item[-14:])
    except: 
        pass 
    
    return

def decorde_par(item):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
        timestamp = pd.to_datetime(np.loadtxt('timestamp.info',dtype=str)[0])
        try:
            d = parse_metar_to_dataframe(item.values[0][52:], year=timestamp.year, month=timestamp.month)
            return d
        except:
            print('Error with METAR: ', item.values[0][52:])


class ASOS():

    def __init__(self):
        self.initialized = True

    def grab_datafiles(self,station = None,parallel=True,overwrite=True):

        if station is None:
            print('please provide a station')
            
        from ftplib import FTP
        import os

        ftp =  FTP('ftp.ncdc.noaa.gov') # ftp access to ncdc.noaa.gov
        ftp.login() # anonymous ftp login
        ftp.cwd('pub/data/asos-fivemin/') # change directory to asos-onemin
        dirs = ftp.nlst() 
        dirs.sort()
        dirs = dirs[0:-9]
        file_list = []
        sub = station
        for j in dirs:
            ftp.cwd('/pub/data/asos-fivemin/' + j)
            data_files = ftp.nlst()
            for s in filter (lambda x: sub in x, data_files): file_list.append('ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/'+ j + '/' + s)

        ftp.close()

        url_strs = file_list
        self.urls = url_strs
        ####################################################
        if os.path.isdir('./raw_data/') is False:
            os.mkdir('./raw_data/')

        if parallel:
            import multiprocessing as mp
            pool = mp.Pool(processes=2)
            for _ in tqdm(pool.imap_unordered(download_par, url_strs), total=len(url_strs)):
                  pass
            pool.close()
            pool.join()
        else:
            for item in tqdm(url_strs):
                ff=wget.download(item,'./raw_data/'+item[-14:])

    def process_file(self,filename=None,parallel=True,save=True):
        if os.path.isdir('./decoded_data/') is False:
            os.mkdir('./decoded_data/')

        df = pd.read_csv(filename,header=None)
        # Pull the timestamp from the filename
        timestamp = datetime.datetime.strptime(filename[-10:], '%Y%m.dat')

        iterrows = list(df.iterrows())
        index, rows = zip(*iterrows)

        logfile = open('timestamp.info', "w")
        logfile.write(str(timestamp))
        logfile.close()

        df_list = []

        if parallel:
            import multiprocessing as mp
            pool = mp.Pool(processes=10)
            for d in tqdm(pool.imap(decorde_par, rows), total=len(rows)):
                  df_list.append(d)
            pool.close()
            pool.join()
        else:
            timestamp = pd.to_datetime(np.loadtxt('timestamp.info',dtype=str)[0])
            for item in tqdm(rows):
                try:
                    df_list.append(parse_metar_to_dataframe(item.values[0][52:], year=timestamp.year, month=timestamp.month))
                except:
                    print('Error with METAR: ', item.values[0][52:])

        df_master = pd.concat(df_list)
        df_master = df_master.sort_values('date_time')

        if save:
            df_master.to_csv('./decoded_data/' + filename[-10:-4] + '.csv')

        self.df_master = df_master 


    def process_all_files(self):
        files = glob.glob('./raw_data/*.dat')
        files.sort()
        for i in files[151:]:
            print(i)
            if i == './raw_data/KIMT201608.dat':
                continue
            self.process_file(filename=i)

In [ ]:
#grab and download 
asos = ASOS()
asos.grab_datafiles(station='KIMT')

In [ ]:
!ls ./raw_data/

In [ ]:
#Process
asos.process_all_files()

In [ ]:
#concat all files
files = glob.glob('./decoded_data/*')
files.sort()
df_list = []
for i in files:
    df_list.append(pd.read_csv(i))

df = pd.concat(df_list)
df = df.drop(columns=['air_pressure_at_sea_level','low_cloud_type','high_cloud_level', 'highest_cloud_type','highest_cloud_level','low_cloud_level', 'medium_cloud_level', 'high_cloud_type','current_wx1', 'current_wx2', 'current_wx3','Unnamed: 0','station_id','latitude','longitude','cloud_coverage','past_weather','medium_cloud_type','eastward_wind','northward_wind','past_weather2'])
df = df.dropna(how='any')

In [ ]:
#calc some params.
altim = df.altimeter.values * units.in_Hg
height = df.elevation.values * units.meter
temp = df.air_temperature.values *units.degC
dew = df.dew_point_temperature.values *units.degC
slp = mpcalc.altimeter_to_sea_level_pressure(altim,height,temp)
p = mpcalc.altimeter_to_station_pressure(altim,height)
p = p.to(units.hectopascal)
slp = slp.to(units.hectopascal)
slp = np.asarray(slp,dtype=float)
p = np.asarray(p,dtype=float)

In [ ]:
#grab only weather times
df_timeseries = pd.DataFrame({'wind_direction':df.wind_direction.values,'wind_speed':df.wind_speed.values,'air_temperature':df.air_temperature.values,'dew_point_temperature':df.dew_point_temperature.values,'present_weather':df.present_weather.values,'sea_level_pressure':slp,'pressure':p},index=pd.to_datetime(df.date_time.values))
date_indexs = pd.date_range(df_timeseries.index.values.min(),df_timeseries.index.values.max(),freq='300S')
df_timeseries = df_timeseries[~df_timeseries.index.duplicated()]
df_timeseries = df_timeseries.reindex(index=date_indexs)
df_nowx = df_timeseries.where(df_timeseries.present_weather == 0).dropna(how='all')
df_wx = df_timeseries.where(df_timeseries.present_weather > 0).dropna(how='all')

In [ ]:
#calc wet bulb when there is weather 
zipped = list(zip(df_wx.pressure.values,df_wx.air_temperature.values,df_wx.dew_point_temperature.values,np.arange(0,len(df_wx.dew_point_temperature.values))))

def wet_bulb_par(item):
    try:
        tw = mpcalc.wet_bulb_temperature(item[0:1]*units.hectopascal, item[1:2]*units.degC, item[2:3]*units.degC)
    except:
        tw = np.nan
    return [tw,item[3]]

import multiprocessing as mp
from tqdm import tqdm
pool = mp.Pool(processes=24)
tw_list = []
for d in tqdm(pool.imap(wet_bulb_par, zipped), total=len(zipped)):
      tw_list.append(d)
pool.close()
pool.join()

lst1, lst2 = zip(*tw_list)
ttt = np.zeros(len(lst1))
#force the correct index 
for i,ii in enumerate(lst1):
    ttt[lst2[i]] = np.asarray(lst1[i])
    
df_wx['wet_bulb'] = pd.Series(ttt, index=df_wx.index)

In [ ]:
# Split data
snow = [70,71,72,73,74,75,76,77,78,79,85,86]
rain = [50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,80,81,82]

df_list = []
for i in snow:
    df_list.append(df_wx.where(df_wx.present_weather == i).dropna(how='all'))
    
df_snow=pd.concat(df_list)

df_list = []
for i in rain:
    df_list.append(df_wx.where(df_wx.present_weather == i).dropna(how='all'))
    
df_rain=pd.concat(df_list)

In [ ]:
#take a gander at data 
%config InlineBackend.figure_format = 'retina'
plt.hist(df_snow.present_weather,bins=np.arange(50,90),color='dodgerblue',edgecolor='k',label='Frozen Precip.')
plt.hist(df_rain.present_weather,bins=np.arange(50,90),color='orangered',edgecolor='k',label='Liquid Precip.')
plt.semilogy()
plt.legend(loc=1)
plt.title('KIMT, Iron Mountain MI, 2005 - 2019')
plt.xlabel('Present Weather Code')
plt.ylabel('Count')
plt.text(48.5,5e4,'n_samples : {}'.format(df_snow.present_weather.shape[0]),color='dodgerblue')
plt.text(48.5,3e4,'n_samples : {}'.format(df_rain.present_weather.shape[0]),color='orangered')

### Part 2: Logistic Classification 

In [ ]:
!git clone https://github.com/dopplerchase/ATMS-597-SP-2020.git
rain = pd.read_csv('./ATMS-597-SP-2020/ATMS-597-SP-2020-Project-5/RAIN_KIMT.csv')
snow = pd.read_csv('./ATMS-597-SP-2020/ATMS-597-SP-2020-Project-5/SNOW_KIMT.csv')

data = pd.concat([rain,snow])
data.tail()

In [ ]:
# Check which present weather codes are in our data
data['present_weather'].value_counts()

In [ ]:
# Change frozen/liquid precip to read as such 
data['present_weather'].replace(to_replace=[61.0, 63.0, 65.0, 66.0, 67.0, 71.0, 73.0, 75.0], value= ['Liquid', 'Liquid', 'Liquid', 'Liquid', 'Liquid', 'Frozen', 'Frozen', 'Frozen'], inplace=True)

# Rename present weather column and drop unecessary columns
data.rename(columns={'present_weather': 'Precipitation_Type'}, inplace = True)
data.drop('Unnamed: 0', axis = 1, inplace= True)
data.head()

In [ ]:
data['Precipitation_Type'].value_counts()

In [ ]:
# Check for NaNs
data.isnull().sum()

In [ ]:
# Drop rows with a Nan
data.dropna(axis=0, how='any', inplace=True)
data.isnull().sum()

In [ ]:
data['Precipitation_Type'].value_counts()

In [ ]:
# Set liquid precip equal to 1, frozen to 0 
data.Precipitation_Type = [1 if each == 'Liquid' else 0 for each in data.Precipitation_Type]
data.head()

In [ ]:
#split dataset into features and target variable
cols = ['wind_direction', 'wind_speed',	'air_temperature', 'dew_point_temperature',	'sea_level_pressure',	'pressure',	'wet_bulb']
X = data[cols] #features
y = data.Precipitation_Type # Target variable

# Scale X matrix
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

# Build and fit logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

In [ ]:
# Construct confusion matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

# Plot confusion matrix
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
# Some statistics

# Build array of length y_test with 48% ones, 52% zeroes for climo y_pred
prob_array = np.zeros([100],dtype=int)
prob_array[0:48] = 1
np.random.shuffle(prob_array)
y_pred2 = np.random.choice(prob_array,size=y_test.shape[0])

# create variables of model and climo brier scores to calc brier skill score
bs_m = metrics.brier_score_loss(y_test, y_pred)
bs_c = metrics.brier_score_loss(y_test, y_pred2)
bss = ((bs_c - bs_m)/bs_c)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Logistic Regression Brier Score:", metrics.brier_score_loss(y_test, y_pred))
print("Climatology Brier Score:", metrics.brier_score_loss(y_test, y_pred2))
print("Brier Skill Score:", bss)

In [ ]:
# Plot ROC curve 
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="Logistic Regression, auc="+str(auc))
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

### Gaussian Naive Bayes Model

In [ ]:
# split dataset into features and target variable
cols = ['wind_direction', 'wind_speed',	'air_temperature', 'dew_point_temperature',	'sea_level_pressure',	'pressure',	'wet_bulb']
X2 = data[cols] #features
y = data.Precipitation_Type # Target variable

# scale
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X2 = scaler.fit_transform(X2)

# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X2_train,X2_test,y_train,y_test = train_test_split(X2,y,test_size=0.3,random_state=0)

# creating Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X2_train,y_train)
gnb_y_pred = gnb.predict(X2_test)

In [ ]:
prob_array = np.zeros([100],dtype=int)
prob_array[0:49] = 1
np.random.shuffle(prob_array)
clim_y_pred = np.random.choice(prob_array,size=y_test.shape[0])

In [ ]:
# Construct confusion matrix
from sklearn import metrics
gnb_cnf_matrix = metrics.confusion_matrix(y_test, gnb_y_pred)

# Plot confusion matrix
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(gnb_cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
accuracy = metrics.accuracy_score(y_test, gnb_y_pred)
brier_score = metrics.brier_score_loss(y_test, gnb_y_pred)
brier_score_climo = metrics.brier_score_loss(y_test, clim_y_pred)
print("Accuracy:", accuracy)
print("Brier Score:", brier_score)
print("Brier Score (Climo):", brier_score_climo)
print("Brier Skill Score:", 1 - brier_score/brier_score_climo)